# Quant Trading Strategies Final Project



In [1]:
import pandas as pd
import numpy as np
import requests 
import datetime as dt
import typing
from bs4 import BeautifulSoup
import statsmodels.formula.api as smf
import time

In [15]:
trade_transactions = pd.read_csv('C:/Users/dcste/OneDrive/Economics_Research/Economics_Research/trade_transactions.csv').drop(columns=['Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19'])
trade_transactions = trade_transactions[trade_transactions.type != 'Exchange']

In [63]:
trade_transactions

,transaction_date,disclosure_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,party,state,industry,sector
0,2/16/2023,2/16/2023,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Gary C. Peters,https://efdsearch.senate.gov/search/view/ptr/6...,NaN,NaN,NaN,NaN
1,1/23/2023,2/14/2023,Joint,GOLD,Barrick Gold Corporation Common Stock (BC),Stock,Buy,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
2,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sell,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
3,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sell,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
4,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Buy,"$100,001 - $250,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8252,8/17/2012,8/17/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
8253,8/16/2012,8/16/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Pat Roberts,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
8254,8/15/2012,8/15/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Rob Portman,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
8255,8/2/2012,8/2/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Thomas R. Carper,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN


In [25]:
trade_transactions.type.unique()

array([nan, 'Purchase', 'Sale (Partial)', 'Sale (Full)'], dtype=object)

In [28]:
def filter_trade_type(trade_type:str):
    if trade_type == 'Purchase':
        trade_type = 'Buy'
    elif trade_type == 'Sale (Partial)':
        trade_type = 'Sell'
    elif trade_type == 'Sale (Full)':
        trade_type = 'Sell'
    return trade_type

In [64]:
trade_transactions['type'] = trade_transactions['type'].apply(filter_trade_type)

In [66]:
trade_transactions1  = trade_transactions[(trade_transactions.type == 'Buy')|(trade_transactions.type == 'Sell')]

In [69]:
trade_transactions1

,transaction_date,disclosure_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,party,state,industry,sector
1,1/23/2023,2/14/2023,Joint,GOLD,Barrick Gold Corporation Common Stock (BC),Stock,Buy,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
2,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sell,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
3,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sell,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
4,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Buy,"$100,001 - $250,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
5,1/13/2023,2/14/2023,Joint,UAA,"Under Armour, Inc. Class A Common Stock",Stock,Buy,"$15,001 - $50,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Apparel,Consumer Non-Durables
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8021,1/29/2014,1/31/2014,Spouse,C,"Citigroup, Inc. (NYSE)",NaN,Sell,"$1,001 - $15,000",--,Susan M. Collins,https://efdsearch.senate.gov/search/view/ptr/5...,Republican,ME,Major Banks,Finance
8022,1/28/2014,1/31/2014,Spouse,EBAY,eBay Inc. (NASDAQ),NaN,Sell,"$1,001 - $15,000",--,Susan M. Collins,https://efdsearch.senate.gov/search/view/ptr/5...,Republican,ME,Business Services,Miscellaneous
8023,1/24/2014,1/31/2014,Spouse,FB,"Facebook, Inc. (NASDAQ)",NaN,Buy,"$1,001 - $15,000",--,Susan M. Collins,https://efdsearch.senate.gov/search/view/ptr/5...,Republican,ME,"Computer Software: Programming, Data Processing",Technology
8024,1/24/2014,1/31/2014,Spouse,CRM,"Salesforce.com, Inc (NYSE)",NaN,Buy,"$1,001 - $15,000",--,Susan M. Collins,https://efdsearch.senate.gov/search/view/ptr/5...,Republican,ME,Computer Software: Prepackaged Software,Technology


In [34]:
trade_transactions['type'] = trade_transactions['type'].apply(filter_trade_type)

In [39]:
trade_transactions.type.unique()

array([nan, 'Buy', 'Sell'], dtype=object)

In [61]:
trade_transactions

,transaction_date,disclosure_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,party,state,industry,sector
0,2/16/2023,2/16/2023,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Gary C. Peters,https://efdsearch.senate.gov/search/view/ptr/6...,NaN,NaN,NaN,NaN
1,1/23/2023,2/14/2023,Joint,GOLD,Barrick Gold Corporation Common Stock (BC),Stock,Buy,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
2,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sell,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
3,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sell,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
4,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Buy,"$100,001 - $250,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8252,8/17/2012,8/17/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
8253,8/16/2012,8/16/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Pat Roberts,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
8254,8/15/2012,8/15/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Rob Portman,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
8255,8/2/2012,8/2/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Thomas R. Carper,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
